# Feature Engineering w/ h2o.ai | Word2Vec emp_title

Using Lending Club's dataset from 2015 and prior, there is reason to believe that the emp_title field holds predictive information which we can feature engineer. The Word2Vec algorithm helps find synonyms of employment titles, and transforms the data to vectors.

Read more about Word2Vec here:

http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/word2vec.html

In [1]:
!pip install requests
!pip install tabulate
!pip install scikit-learn
!pip install colorama
!pip install future

In [73]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,50 mins 24 secs
H2O cluster version:,3.16.0.3
H2O cluster version age:,6 days
H2O cluster name:,H2O_from_python_darrenklee_zw6bx3
H2O cluster total nodes:,1
H2O cluster free memory:,1.591 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [74]:
import pandas as pd
import numpy as np

In [98]:
path = "/Users/darrenklee/Desktop/DS-Project/emp_title_loan_status_data.csv"

df = h2o.import_file(path=path)[:,1:3].ascharacter()

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [99]:
df.head()

emp_title,loan_status
Emergency Department technician,0
Corporate Insurance,0
Teacher,0
Regional Sales Direcdtor,1
hvac technician,1
Lead manufacturing,0
Engineer,0
Contract Specialist,0
Systems Support Engineer,0
Application Analyst,0


In [129]:
df.describe()

Rows:183660
Cols:2




,emp_title,loan_status
type,string,string
mins,NaN,NaN
mean,NaN,NaN
maxs,NaN,NaN
sigma,NaN,NaN
zeros,0,0
missing,0,0
0,Emergency Department technician,0
1,Corporate Insurance,0
2,Teacher,0


In [101]:
import h2o
h2o.init()
from h2o.estimators.word2vec import H2OWord2vecEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,1 hour 13 mins
H2O cluster version:,3.16.0.3
H2O cluster version age:,6 days
H2O cluster name:,H2O_from_python_darrenklee_zw6bx3
H2O cluster total nodes:,1
H2O cluster free memory:,1.510 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [102]:
STOP_WORDS = ["ax","i","you","edu","s","t","m","subject","can","lines","re","what",
               "there","all","we","one","the","a","an","of","or","in","for","by","on",
               "but","is","in","a","not","with","as","was","if","they","are","this","and","it","have",
               "from","at","my","be","by","not","that","to","from","com","org","like","likes","so"]

In [103]:
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

In [104]:
def predict(df,w2v, gbm):
    words = tokenize(h2o.H2OFrame(df).ascharacter())
    df_vec = w2v.transform(words, aggregate_method="AVERAGE")
    print(gbm.predict(test_data=job_title_vec))

In [105]:

print("Break job titles into sequence of words")
words = tokenize(df["emp_title"])

Break job titles into sequence of words


In [106]:
print("Build word2vec model")
w2v_model = H2OWord2vecEstimator(sent_sample_rate = 0.0, epochs = 10)
w2v_model.train(training_frame=words)

Build word2vec model
word2vec Model Build progress: |██████████████████████████████████████████| 100%


In [138]:
print("Sanity check - find synonyms for the word 'painter'")
w2v_model.find_synonyms("painter", count = 10)

Sanity check - find synonyms for the word 'teacher'


OrderedDict([(u'glazier', 0.7945498824119568),
             (u'plumber', 0.7230222821235657),
             (u'carpenter', 0.6847513914108276),
             (u'blaster', 0.667998194694519),
             (u'electrician', 0.6575730443000793),
             (u'assembler', 0.6496596932411194),
             (u'pipefitter', 0.6402847766876221),
             (u'craftsman', 0.6321462392807007),
             (u'roofer', 0.6291592121124268),
             (u'working', 0.6070107817649841)])

In [125]:
print("Calculate a vector for each emp_title")
emp_title_vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")

Calculate a vector for each emp_title


In [152]:
data2 = df.cbind(emp_title_vecs)
data2.shape

(183660, 102)

In [126]:
print("Prepare training&validation data (keep only job titles made of known words)")
valid_emp_titles = ~ emp_title_vecs["C1"].isna()
data = df[valid_emp_titles,:].cbind(emp_title_vecs[valid_emp_titles,:])
data_split = data.split_frame(ratios=[0.8])

Prepare training&validation data (keep only job titles made of known words)


In [144]:
data

emp_title,loan_status,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
Emergency Department technician,0,-0.237751,-0.0729316,0.0894989,-0.139048,-0.182811,0.239338,0.00941441,0.0824646,-0.332516,-0.344786,0.227957,-0.193663,0.14162,0.0782099,-0.317593,0.100721,0.0503919,-0.101892,-0.380651,-0.155539,0.211975,-0.160445,0.0343821,0.114849,-0.266019,-0.17087,-0.0313839,0.0446568,-0.163996,0.0782537,0.160748,-0.0504089,-0.128587,0.406312,0.119234,-0.0112055,0.132531,0.0268563,-0.090749,0.220527,-0.00394953,0.443641,-0.169868,-0.156508,0.0884958,-0.0816054,0.189869,0.0377647,0.0448495,0.0268733,-0.0199931,-0.514803,0.117256,-0.0347837,-0.330134,0.241807,-0.00643388,-0.573299,-0.128931,0.0943115,-0.218511,0.3349,0.163035,-0.114219,0.121819,0.0171678,0.0486514,-0.173862,0.149217,0.356777,0.0638926,-0.0508236,-0.249609,-0.118606,0.268285,0.135556,-0.153362,-0.185457,0.129406,0.199583,-0.179948,-0.0128305,0.109334,0.0762421,-0.148976,0.162148,0.184708,0.0413629,-0.0431332,0.146621,0.0263277,-0.0819337,0.0930371,-0.240905,-0.0900113,0.0637557,-0.258116,-0.550784,-0.072068,0.314838
Corporate Insurance,0,0.136679,-0.144567,-0.219635,0.176496,0.0589623,0.115669,0.0956495,0.240872,-0.316507,-0.125302,0.225058,0.160342,-0.163577,0.0327881,-0.153917,-0.155174,0.0310965,-0.198121,0.23992,-0.1159,0.349041,-0.119669,-0.147016,0.345455,-0.477145,0.112978,0.45149,-0.3525,0.0841605,0.0959851,0.154836,0.151768,0.365497,-0.146185,0.241859,-0.0574159,0.219442,0.165528,-0.0866035,-0.408591,0.327945,0.772904,-0.00804143,-0.36964,0.229165,-0.0881138,0.115113,0.167157,0.128234,-0.280985,-0.213696,0.116153,-0.52627,0.225373,-0.333489,0.0850746,0.123853,-0.105959,0.343277,-0.0378136,0.306728,0.137913,0.130111,0.390109,-0.0933352,-0.00156192,-0.131371,-0.284189,0.190074,-0.178177,-0.35391,-0.0950503,0.353733,-0.348374,0.0182431,0.117761,0.14017,0.120819,0.275642,0.300094,0.273035,-0.416597,0.177683,-0.145328,0.0676711,-0.0101007,-0.0448569,-0.0682249,0.62681,-0.106503,0.00962499,-0.216761,-0.132466,0.0869347,0.115855,-0.268649,-0.298816,0.18687,0.140972,0.203839
Teacher,0,-0.420405,0.337638,-0.0418651,-0.713847,-0.55096,0.108866,0.171319,-0.330193,-0.873747,0.549869,0.0328137,-0.0576892,0.365431,-0.00504889,-0.421131,-0.0436886,-0.313658,0.309085,0.411751,0.0619689,0.0512558,-0.0219861,-0.416153,-0.897691,0.146338,-0.54097,-0.681878,-0.821806,0.421511,-0.0488635,0.644557,0.000863975,0.373951,-0.39089,-0.479522,-0.7585,0.164389,0.116286,-0.0350933,-0.486592,-0.781227,0.199507,-0.556219,-0.509122,0.499199,-0.143516,0.181543,-0.160599,0.295796,-0.523104,-0.217884,-0.264076,0.592124,-1.00842,-0.158945,0.221271,-0.50484,0.758098,-0.0792309,0.420502,-0.549695,-0.46556,-0.305701,-0.659274,-0.485684,-0.330519,0.266906,-0.631643,0.483587,0.674028,0.192482,-0.184162,-0.38473,0.122553,-0.371431,-0.415303,0.0378919,-0.00271226,-0.412394,0.0226645,-0.179652,-0.0218947,0.131175,0.730714,-0.695195,-0.416138,-0.524377,-0.219182,0.014905,-0.336285,0.520902,0.33412,-0.838555,0.173891,-0.727107,-0.246951,-0.474045,-0.0591594,0.952233,0.324939
Regional Sales Direcdtor,1,0.398176,0.0716235,-0.295504,0.082468,0.415381,-0.258422,0.184185,-0.0737997,0.490681,-0.207707,0.371796,-0.22867,-0.102725,-0.0743461,-0.311047,0.0111619,0.256179,0.104013,0.0728996,-0.0256856,0.0853533,-0.00739908,0.0527599,0.116054,-0.281037,-0.0478498,0.229697,-0.133704,0.105423,0.229776,-0.000675047,0.182249,0.144669,0.405736,0.358409,-0.125551,0.0174678,0.251763,-0.354572,-0.0957421,-0.24682,-0.0757709,-0.149326,-0.0373113,-0.0524477,0.0217119,0.289294,0.163319,0.177032,-0.066881,-0.06129,0.036511,-0.194871,0.0516559,0.0516741,0.481722,-0.269713,0.0924776,0.0516088,0.2

In [145]:
# export data to csv file!
import pandas as pd
import numpy as np

In [147]:
data.shape

(180614, 102)

In [131]:
print("Build a basic GBM model")
gbm_model = H2OGradientBoostingEstimator()
gbm_model.train(x = emp_title_vecs.names,
                y="loan_status", 
                training_frame = data_split[0], 
                validation_frame = data_split[1])

Build a basic GBM model
gbm Model Build progress: | (failed)


EnvironmentError: Job with key $03017f00000132d4ffffffff$_b660ce92acd31ba66461f5324c7e4f12 failed with an exception: DistributedException from /127.0.0.1:54321: 'Operation not allowed on string vector.', caused by java.lang.IllegalArgumentException: Operation not allowed on string vector.
stacktrace: 
DistributedException from /127.0.0.1:54321: 'Operation not allowed on string vector.', caused by java.lang.IllegalArgumentException: Operation not allowed on string vector.
	at water.MRTask.getResult(MRTask.java:478)
	at water.MRTask.getResult(MRTask.java:486)
	at water.MRTask.doAll(MRTask.java:390)
	at water.MRTask.doAll(MRTask.java:377)
	at water.MRTask.doAll(MRTask.java:376)
	at hex.tree.SharedTree.getInitialValue(SharedTree.java:981)
	at hex.tree.gbm.GBM.access$1800(GBM.java:24)
	at hex.tree.gbm.GBM$GBMDriver.initializeModelSpecifics(GBM.java:177)
	at hex.tree.SharedTree$Driver.computeImpl(SharedTree.java:353)
	at hex.ModelBuilder$Driver.compute2(ModelBuilder.java:206)
	at water.H2O$H